In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import geopy
from datetime import datetime, date
import os
import sys
import json
import logging  
#from keplergl import KeplerGl
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")
logger=logging.getLogger(__name__)
logger.setLevel(level=logging.DEBUG)

base_dir = os.getcwd()

data_dir = os.path.join(base_dir, "../../data/wastewaterscan/")

output_dir = os.path.join(base_dir, "../../data/wastewaterscan/output/")


Plant location

In [112]:
plantfiles= "plants.json"
filepath = os.path.join(data_dir, plantfiles)
f = open(filepath)
data = json.load(f)

#plant_json= pd.read_json(filepath,typ='frame', orient='records')
#plant_json= pd.json_normalize(data["plants"] )
plant_json= pd.DataFrame.from_dict(data["plants"])
#plant_json.rename(columns={"point":"point_obj", "polygon":"polygon_obj"}, inplace=True)
plant_json.dropna(inplace=True,subset=['point'])
plant_json.info()
plant_json.head()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 0 to 193
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               183 non-null    object
 1   city             183 non-null    object
 2   state            183 non-null    object
 3   name             183 non-null    object
 4   site_name        183 non-null    object
 5   place_name       183 non-null    object
 6   counties_served  183 non-null    object
 7   flow_rate        0 non-null      object
 8   sewershed_pop    183 non-null    int64 
 9   zipcode          181 non-null    object
 10  point            183 non-null    object
 11  polygon          104 non-null    object
dtypes: int64(1), object(11)
memory usage: 18.6+ KB


,id,city,state,name,site_name,place_name,counties_served,flow_rate,sewershed_pop,zipcode,point,polygon
0,74d026bd-cb5d-411b-a57b-b70f422f8fc6,Richmond,California,"West Contra Costa County, CA",West County Wastewater District,"Richmond, California",[06013],None,100000,94801,"{'type': 'Point', 'coordinates': [-122.3198807...","{'type': 'Polygon', 'coordinates': [[[-122.313..."
1,e136c15f-df7c-4b04-8973-103a6f095682,Union Beach,New Jersey,"Union Beach, NJ",Bayshore Regional Sewerage Authority,"Union Beach, New Jersey",[34025],None,100000,07735,"{'type': 'Point', 'coordinates': [-74.17774894...",None
2,07ced721-3d2c-4f05-a24a-8d4bf937177b,Silicon Valley,California,"Redwood City, CA",Silicon Valley Clean Water,"Silicon Valley, California",[06081],None,199000,94065,"{'type': 'Point', 'coordinates': [-122.2283798...","{'type': 'Polygon', 'coordinates': [[[-122.244..."
3,b723ae76-6e2e-4f45-954e-094c535590c2,Napa,California,"Napa, CA",Soscol Water Recycling Facility,"Napa, California",[06055],None,83300,94558,"{'type': 'Point', 'coordinates': [-122.3052443...","{'type': 'Polygon', 'coordinates': [[[-122.358..."
5,173560dc-6025-441c-b9d2-a6c0525008c0,Hollywood,Maryland,"Hollywood, MD",Marlay Taylor Water Reclamation Facility,"Hollywood, Maryland",[24037],None,55000,20653,"{'type': 'Point', 'coordinates': [-76.40913246...","{'type': 'Polygon', 'coordinates': [[[-76.4775..."


In [127]:
def new_point(pointfield):
    coords = pointfield["coordinates"]
    lat = coords[1]
    lon = coords[0]
    return Point(lon,lat)

points = plant_json['point'].apply(lambda x: new_point(x))
plant_json.info()
plant_json.head()
plant_gdf = gpd.GeoDataFrame(plant_json, 
    geometry = points, 
    crs = 'EPSG:4326')
plant_gdf['site_name']=plant_gdf['site_name'].astype(str)
plant_gdf['Latitude'] = plant_gdf.geometry.y
plant_gdf['Longitude'] = plant_gdf.geometry.x
plant_gdf.info()
plant_gdf.head()

plant_names = plant_gdf['site_name'].unique().tolist()
plant_names = pd.Series(plant_names)
plant_names = plant_names.rename("site_name", inplace=True).to_frame()

plant_names.info()
plant_names.head()


<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 0 to 193
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               183 non-null    object
 1   city             183 non-null    object
 2   state            183 non-null    object
 3   name             183 non-null    object
 4   site_name        183 non-null    object
 5   place_name       183 non-null    object
 6   counties_served  183 non-null    object
 7   flow_rate        0 non-null      object
 8   sewershed_pop    183 non-null    int64 
 9   zipcode          181 non-null    object
 10  point            183 non-null    object
 11  polygon          104 non-null    object
dtypes: int64(1), object(11)
memory usage: 18.6+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 183 entries, 0 to 193
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   id         

,site_name
0,West County Wastewater District
1,Bayshore Regional Sewerage Authority
2,Silicon Valley Clean Water
3,Soscol Water Recycling Facility
4,Marlay Taylor Water Reclamation Facility


Data

In [115]:
filename ="WWSCAN_CA_plants_all_targets_20240502.csv"
filepath = os.path.join(data_dir, filename)
wss_df = pd.read_csv(
    filepath, delimiter=",", 
    )
wss_df['collection_date'] = pd.to_datetime(wss_df['collection_date'])

wss_df.head()

,sample_id,collection_date,bcov_recovery,bcov_recovery_lci,bcov_recovery_uci,city,state,name,site_name,sewershed_pop,...,Rota_gc_g_dry_weight_uci,C_auris_gc_g_dry_weight,C_auris_gc_g_dry_weight_lci,C_auris_gc_g_dry_weight_uci,HAV_gc_g_dry_weight,HAV_gc_g_dry_weight_lci,HAV_gc_g_dry_weight_uci,HPIV_gc_g_dry_weight,HPIV_gc_g_dry_weight_lci,HPIV_gc_g_dry_weight_uci
0,COD110144,2021-07-29,1.804,1.699,1.902,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,COD110138,2021-07-30,2.076,1.926,2.292,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COD110132,2021-08-03,8.111,7.805,8.436,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,COD110120,2021-08-04,1.039,0.973,1.095,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COD110114,2021-08-05,1.083,1.052,1.107,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Pathogens

In [169]:
wss_pathogens_base_names = wss_df.columns.values.tolist()

wss_pathogens_base_names = filter(lambda x: 'gc_g_dry_weight_uci' in x, wss_pathogens_base_names) 
wss_pathogens_base_names = map(lambda x: x.replace('_gc_g_dry_weight_uci','') ,wss_pathogens_base_names)
wss_pathogens_base_names =list(wss_pathogens_base_names)
wss_pathogens_dry_names = map(lambda x: x + '_gc_g_dry_weight' ,wss_pathogens_base_names)
wss_pathogens_dry_names = list(wss_pathogens_dry_names)
wss_mpox_names  = ['MPXV_G2R_G_gc_g_dry_weight', 'MPXV_G2R_WA_gc_g_dry_weight']
wss_flu_names = ['Influenza_A_gc_g_dry_weight', 'Influenza_B_gc_g_dry_weight']
wss_pathogens_base_names

['N_Gene',
 'S_Gene',
 'PMMoV',
 'RSV',
 'Influenza_A',
 'HMPV_4',
 'Influenza_B',
 'MPXV_G2R_G',
 'Noro_G2',
 'MPXV_G2R_WA',
 'EVD68',
 'Rota',
 'C_auris',
 'HAV',
 'HPIV']

In [170]:
wss_df['pathogens_dry_weight_sum'] = wss_df[wss_pathogens_dry_names].fillna(0).sum(axis=1)
wss_df['pathogens_dry_weight_max'] = wss_df[wss_pathogens_dry_names].fillna(0).max(axis=1)

wss_df['mpox_dry_weight_sum'] = wss_df[wss_mpox_names].fillna(0).sum(axis=1)
wss_df['mpox_dry_weight_max'] = wss_df[wss_mpox_names].fillna(0).max(axis=1)
wss_df['influenza_dry_weight_sum'] = wss_df[wss_flu_names].fillna(0).sum(axis=1)
wss_df['influenza_dry_weight_max'] = wss_df[wss_flu_names].fillna(0).max(axis=1)
wss_df.head()

,sample_id,collection_date,bcov_recovery,bcov_recovery_lci,bcov_recovery_uci,city,state,name,site_name,sewershed_pop,...,HPIV_gc_g_dry_weight,HPIV_gc_g_dry_weight_lci,HPIV_gc_g_dry_weight_uci,pathogens_dry_weight,pathogens_dry_weight_sum,pathogens_dry_weight_max,mpox_dry_weight_sum,mpox_dry_weight_max,influenza_dry_weight_sum,influenza_dry_weight_max
0,COD110144,2021-07-29,1.804,1.699,1.902,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,3.867427e+08,3.867427e+08,3.867297e+08,0.0,0.0,0.0,0.0
1,COD110138,2021-07-30,2.076,1.926,2.292,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,7.827964e+08,7.827964e+08,7.827130e+08,0.0,0.0,0.0,0.0
2,COD110132,2021-08-03,8.111,7.805,8.436,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,2.853652e+10,2.853652e+10,2.853627e+10,0.0,0.0,0.0,0.0
3,COD110120,2021-08-04,1.039,0.973,1.095,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,5.578497e+10,5.578497e+10,5.578457e+10,0.0,0.0,0.0,0.0
4,COD110114,2021-08-05,1.083,1.052,1.107,CODIGA,California,"CODIGA, Stanford, CA",CODIGA,10000,...,NaN,NaN,NaN,3.101570e+09,3.101570e+09,3.101456e+09,0.0,0.0,0.0,0.0


In [116]:
wss_names = wss_df[['name','site_name', 'city','state']].drop_duplicates()
wss_names['full_name'] = wss_names['name'] + ", US"
wss_names.to_csv(os.path.join(output_dir, "plants_names.csv"), index=False)
wss_names['site_name']=wss_names['site_name'].astype(str)
wss_names 

,name,site_name,city,state,full_name
0,"CODIGA, Stanford, CA",CODIGA,CODIGA,California,"CODIGA, Stanford, CA, US"
818,"Coastal, Laguna Niguel, CA",Coastal Treatment Plant,Laguna Niguel,California,"Coastal, Laguna Niguel, CA, US"
1031,"Contra Costa County, CA",Central Contra Costa Sanitary District,Martinez,California,"Contra Costa County, CA, US"
1335,"Davis, CA",City of Davis Wastewater Treatment Plant,Davis,California,"Davis, CA, US"
2302,"Esparto, CA",Esparto Wastewater Treatment Facility,Esparto,California,"Esparto, CA, US"
2515,"Fairfield, CA",Fairfield-Suisun Sewer District,Fairfield,California,"Fairfield, CA, US"
2763,"Fremont, CA",[Fremont Basin] - Raymond A. Boege Alvarado WWTP,Union City,California,"Fremont, CA, US"
2902,"Gilroy, CA",South County Regional Wastewater Authority,Gilroy,California,"Gilroy, CA, US"
4147,"Half Moon Bay, CA",Sewer Authority Mid-Coastside,Half Moon Bay,California,"Half Moon Bay, CA, US"
4434,"Hollister, CA",City of Hollister Domestic Water Recycling Fac...,Hollister,California,"Hollister, CA, US"


In [117]:
matched_plants= plant_names.merge(wss_names, on='site_name', how='inner')
matched_plants.to_csv(os.path.join(output_dir, "plants_matched.csv"), index=False)


In [171]:
base_remove = ["id",'name_x', 'city_x', 'state_x', 
                    'name_y', 'city_y', 'state_y', 'sewershed_pop_x',
                         'counties_served', 'point','polygon',]
for n in wss_pathogens_base_names:
    base_remove.append(f"{n}_gc_g_dry_weight_lci")
    base_remove.append(f"{n}_gc_g_dry_weight_uci")
wss_locations_gdf = plant_gdf.merge(wss_df, on='site_name', how='inner')
wss_locations_gdf .drop(["id",'name_x', 'city_x', 'state_x', 
                    'name_y', 'city_y', 'state_y', 'sewershed_pop_x',
                         'counties_served', 'point','polygon',
                        'N_Gene_gc_g_dry_weight_lci' ,'N_Gene_gc_g_dry_weight_uci',
                        'S_Gene_gc_g_dry_weight_lci' ,'S_Gene_gc_g_dry_weight_uci',
                         'PMMoV_gc_g_dry_weight_lci' ,'PMMoV_gc_g_dry_weight_uci',
                         'RSV_gc_g_dry_weight_lci' ,'RSV_gc_g_dry_weight_uci',
                         'Influenza_A_gc_g_dry_weight_lci' ,'Influenza_A_gc_g_dry_weight_uci',
                         'HMPV_4_gc_g_dry_weight_lci' ,'HMPV_4_gc_g_dry_weight_uci',
                         'Influenza_B_gc_g_dry_weight_lci' ,'Influenza_B_gc_g_dry_weight_uci',
                        'MPXV_G2R_G_gc_g_dry_weight_lci' ,'MPXV_G2R_G_gc_g_dry_weight_uci',
                         'Noro_G2_gc_g_dry_weight_lci' ,'Noro_G2_gc_g_dry_weight_uci',
                         'MPXV_G2R_WA_gc_g_dry_weight_lci' ,'MPXV_G2R_WA_gc_g_dry_weight_uci',
                         'EVD68_gc_g_dry_weight_lci' ,'EVD68_gc_g_dry_weight_uci',
                         'Rota_gc_g_dry_weight_lci' ,'Rota_gc_g_dry_weight_uci',
                         'C_auris_gc_g_dry_weight_lci' ,'C_auris_gc_g_dry_weight_uci',
                         'HAV_gc_g_dry_weight_lci' ,'HAV_gc_g_dry_weight_uci',
                          'HPIV_gc_g_dry_weight_lci' ,'HPIV_gc_g_dry_weight_uci'
                         ], axis=1, inplace=True) 
wss_locations_gdf .info()
wss_locations_gdf .head(60)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23291 entries, 0 to 23290
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   site_name                    23291 non-null  object        
 1   place_name                   23291 non-null  object        
 2   flow_rate                    0 non-null      object        
 3   zipcode                      21780 non-null  object        
 4   geometry                     23291 non-null  geometry      
 5   Latitude                     23291 non-null  float64       
 6   Longitude                    23291 non-null  float64       
 7   sample_id                    23291 non-null  object        
 8   collection_date              23291 non-null  datetime64[ns]
 9   bcov_recovery                23291 non-null  float64       
 10  bcov_recovery_lci            23291 non-null  float64       
 11  bcov_recovery_uci            2329

,site_name,place_name,flow_rate,zipcode,geometry,Latitude,Longitude,sample_id,collection_date,bcov_recovery,...,C_auris_gc_g_dry_weight,HAV_gc_g_dry_weight,HPIV_gc_g_dry_weight,pathogens_dry_weight,pathogens_dry_weight_sum,pathogens_dry_weight_max,mpox_dry_weight_sum,mpox_dry_weight_max,influenza_dry_weight_sum,influenza_dry_weight_max
0,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220510,2022-05-10,1.289,...,NaN,NaN,NaN,7.868723e+08,7.868723e+08,7.864359e+08,0.000000,0.000000,6976.904161,6976.904161
1,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220512,2022-05-12,1.679,...,NaN,NaN,NaN,3.274157e+09,3.274157e+09,3.273103e+09,0.000000,0.000000,20535.195830,20535.195830
2,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220516,2022-05-16,0.679,...,NaN,NaN,NaN,5.339474e+08,5.339474e+08,5.334632e+08,0.000000,0.000000,5480.349985,5480.349985
3,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220517,2022-05-17,0.732,...,NaN,NaN,NaN,9.480382e+08,9.480382e+08,9.477445e+08,0.000000,0.000000,2564.306100,2564.306100
4,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220519,2022-05-19,1.764,...,NaN,NaN,NaN,4.728751e+08,4.728751e+08,4.723031e+08,0.000000,0.000000,7665.078386,7665.078386
5,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220523,2022-05-23,0.372,...,NaN,NaN,NaN,1.351294e+08,1.351294e+08,1.348238e+08,0.000000,0.000000,0.000000,0.000000
6,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220531,2022-05-31,0.439,...,NaN,NaN,NaN,4.475094e+08,4.475094e+08,4.472013e+08,0.000000,0.000000,2723.878143,2723.878143
7,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220602,2022-06-02,0.578,...,NaN,NaN,NaN,2.142347e+09,2.142347e+09,2.141543e+09,0.000000,0.000000,20889.633005,20889.633005
8,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220606,2022-06-06,2.106,...,NaN,NaN,NaN,3.608856e+08,3.608856e+08,3.605423e+08,0.000000,0.000000,3276.647732,3276.647732
9,West County Wastewater District,"Richmond, California",None,94801,POINT (-122.31988 37.97275),37.972746,-122.319881,43-220607,2022-06-07,0.729,...,NaN,NaN,NaN,8.368042e+08,8.368042e+08,8.354329e+08,0.000000,0.000000,6990.971603,6990.971603


In [172]:

wss_locations_gdf.to_file(os.path.join(output_dir, "wwscan.geojson"),driver='GeoJSON')


In [173]:
wss_locations_gdf.drop('geometry', axis=1).to_csv(os.path.join(output_dir, "wwscan.csv"), index=False)